In [1]:
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "3"
sys.path.insert(0, "/root/project/rlt2t")

In [2]:
import pandas as pd
from rlt2t.engines.t2t_engine import T2TEngineCT2, T2TEngineHF
from rlt2t.predictor.predictor import Predictor

In [3]:
input_file = "/root/project/rlt2t/data/t2t/train_2_train/test.json"

In [4]:
df = pd.read_json(input_file, lines=True)
records = df.to_dict('records')

In [5]:
texts = [item['text'] for item in records]
summarys = [item['summary'] for item in records]

In [27]:
t2t_model_paths = [
    # "/root/autodl-tmp/sub-models/uer-large-199-0.1",
    "/root/autodl-tmp/sub-models/uer-large-199-0.2",
    "/root/autodl-tmp/sub-models/idea-bart-xl-0.2",
    # "/root/autodl-tmp/output-models/idea/bart-xl-0.2-rank/ct2",
    #"/root/autodl-tmp/sub-models/idea-bart-xl-0.3", 
    "/root/autodl-tmp/sub-models/uer-base-139-0.1-142",
    #"/root/autodl-tmp/sub-models/uer-large-199-0.1-rank",
    #"/root/autodl-tmp/sub-models/uer-large-199-0.2-rank",
    #"/root/autodl-tmp/sub-models/uer-base-139-0.1-188",
    "/root/autodl-tmp/sub-models/fnlp-base-249-242-503650",
    #"/root/autodl-tmp/sub-models/fnlp-base-249-242-503657",
    #"/root/autodl-tmp/sub-models/idea-bart-xl-0.2-rank",
]

t2t_score_model_paths = [
    #"/root/autodl-tmp/sub-models/uer-large-199-0.1",
    #"/root/autodl-tmp/sub-models/uer-large-199-0.2",
    #"/root/autodl-tmp/sub-models/idea-bart-xl-0.2",
    # "/root/autodl-tmp/output-models/idea/bart-xl-0.2-rank/ct2",
    #"/root/autodl-tmp/sub-models/idea-bart-xl-0.3", 
    #"/root/autodl-tmp/sub-models/uer-base-139-0.1-142",
    "/root/autodl-tmp/sub-models/uer-large-199-0.1-rank",
    "/root/autodl-tmp/sub-models/uer-large-199-0.2-rank",
    #"/root/autodl-tmp/sub-models/uer-base-139-0.1-188",
    #"/root/autodl-tmp/sub-models/fnlp-base-249-242-503650",
    #"/root/autodl-tmp/sub-models/fnlp-base-249-242-503657",
    #"/root/autodl-tmp/output-models/idea/bart-xl-0.2-rank/ct2",
    "/root/autodl-tmp/sub-models/idea-bart-xl-0.2-rank",
    
]

In [28]:
# t2t_model_paths = [
#     "/root/autodl-tmp/sub-models/uer-large-199-0.1",
#     "/root/autodl-tmp/sub-models/uer-large-199-0.2",
#     #"/root/autodl-tmp/sub-models/idea-bart-xl-0.2",
#     #"/root/autodl-tmp/sub-models/idea-bart-xl-0.3", 
#     "/root/autodl-tmp/sub-models/uer-base-139-0.1-142",
#     "/root/autodl-tmp/sub-models/uer-large-199-0.1-rank",
#     "/root/autodl-tmp/sub-models/uer-large-199-0.2-rank",
#     "/root/autodl-tmp/sub-models/uer-base-139-0.1-188",
#     "/root/autodl-tmp/sub-models/fnlp-base-249-242-503650",
#     "/root/autodl-tmp/sub-models/fnlp-base-249-242-503657",
# ]

# t2t_score_model_paths = [
#     "/root/autodl-tmp/sub-models/uer-large-199-0.1",
#     "/root/autodl-tmp/sub-models/uer-large-199-0.2",
#     #"/root/autodl-tmp/sub-models/idea-bart-xl-0.2",
#     #"/root/autodl-tmp/sub-models/idea-bart-xl-0.3", 
#     "/root/autodl-tmp/sub-models/uer-base-139-0.1-142",
#     "/root/autodl-tmp/sub-models/uer-large-199-0.1-rank",
#     "/root/autodl-tmp/sub-models/uer-large-199-0.2-rank",
#     "/root/autodl-tmp/sub-models/uer-base-139-0.1-188",
#     "/root/autodl-tmp/sub-models/fnlp-base-249-242-503650",
#     "/root/autodl-tmp/sub-models/fnlp-base-249-242-503657",
# ]

In [29]:
predictor = Predictor(t2t_model_paths,
                      [],
                      t2t_score_model_paths,
                      beam_size=4,
                      num_hypotheses=4)

In [30]:
output_texts = [item['output'] for item in predictor.predict_v2(texts)]

beam_search_predict...:   0%|          | 0/4 [00:00<?, ?it/s]

predict...:   0%|          | 0/32 [00:00<?, ?it/s]

predict...:   0%|          | 0/32 [00:00<?, ?it/s]

predict...:   0%|          | 0/32 [00:00<?, ?it/s]

predict...:   0%|          | 0/32 [00:00<?, ?it/s]

calculate score...: 0it [00:00, ?it/s]

In [31]:
from rlt2t.utils.evaluate import calculate_scores
from torchmetrics import BLEUScore

In [32]:
def calculate_merge_score(labels, preds):
    cider = calculate_scores(labels, preds, bleu4_rate=0.0).mean()
    bleu_scorer = BLEUScore(4)
    bleu_score = bleu_scorer(preds,
                             [[item] for item in labels]).item()
    print(cider,bleu_score)
    return cider * 2/3 + bleu_score * 1/3

In [33]:
calculate_merge_score(summarys, output_texts)

3.3989094389687735 0.43848171830177307


2.41210019874644